# Sequence Prediction with (Bi-)LSTMs

## Data

We need to get some data. We use the same CoNLL reader function as we had for the Structured Perceptron.

In [ ]:
def read_conll_file(file_name):
    """
    read in a file with CoNLL format:
    word1    tag1
    word2    tag2
    ...      ...
    wordN    tagN

    Sentences MUST be separated by newlines!
    """
    current_words = []
    current_tags = []

    with open(file_name, encoding='utf-8') as conll:
        for line in conll:
            line = line.strip()

            if line:
                word, tag = line.split('\t')
                current_words.append(word)
                current_tags.append(tag)

            else:
                yield (current_words, current_tags)
                current_words = []
                current_tags = []

    # if file does not end in newline (it should...), check whether there is an instance in the buffer
    if current_tags != []:
        yield (current_words, current_tags)

Now we need to do some bookkeeping: collect the set of known words and tags, map both words and tags into integer indices, and keep track of the mapping. We also need two special tokens: one for unknown words, the other one for padding.

In [ ]:
# collect known word tokens and tags
wordset, tagset = set(), set()
train_instances = [(words, tags) for (words, tags) in read_conll_file('tiny_POS_train.data')]
for (words, tags) in train_instances:
    tagset.update(set(tags))
    wordset.update(set(words))

# map words and tags into ints
PAD = '-PAD-'
UNK = '-UNK-'
word2int = {word: i + 2 for i, word in enumerate(sorted(wordset))}
word2int[PAD] = 0  # special token for padding
word2int[UNK] = 1  # special token for unknown words
 
tag2int = {tag: i + 1 for i, tag in enumerate(sorted(tagset))}
tag2int[PAD] = 0
# to translate it back
int2tag = {i:tag for tag, i in tag2int.items()}


def convert2ints(instances):
    result = []
    for (words, tags) in instances:
        # replace words with int, 1 for unknown words
        word_ints = [word2int.get(word, 1) for word in words]
        # replace tags with int
        tag_ints = [tag2int[tag] for tag in tags]
        result.append((word_ints, tag_ints))
    return result        

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g5RQJt7dgBsvNCcYJ-vykjlfQsvTDfadGO61O825kltJegusrJ2GMo


In [ ]:
# get some test data
test_instances = [(words, tags) for (words, tags) in read_conll_file('tiny_POS_test.data')]

# apply integer mapping
train_instances_int = convert2ints(train_instances)
test_instances_int = convert2ints(test_instances)

# separate the words from the tags
train_sentences, train_tags = zip(*train_instances_int) 
test_sentences, test_tags = zip(*test_instances_int) 

print(train_instances[0][0])
print(train_sentences[0])
print(train_instances[0][1])
print(train_tags[0])

Even though RNNs theoretically work for any length input, in practice, we need to make every sentence the same length. We choose the longest training sentence, add some extra, and then pad everything up to that length.

In [ ]:
# get longest training sentence and add 5
MAX_LENGTH = len(max(train_sentences, key=len)) + 5
print(MAX_LENGTH)


## Exercise

Instead of the longest sentence, find the 90th percentile of all training sentence length.

In [ ]:
# Your code here

## Padding

`Keras` provides a function to do the padding for us:

In [ ]:
from keras.preprocessing.sequence import pad_sequences
 
# add special padding at the end of every instance, up to MAX_LENGTH
train_sentences = pad_sequences(train_sentences, maxlen=MAX_LENGTH, padding='post')
test_sentences = pad_sequences(test_sentences, maxlen=MAX_LENGTH, padding='post')
train_tags = pad_sequences(train_tags, maxlen=MAX_LENGTH, padding='post')
test_tags = pad_sequences(test_tags, maxlen=MAX_LENGTH, padding='post')
 
print(train_sentences[0])
print(train_tags[0])

## Model

We use the [functional API](https://keras.io/getting-started/functional-api-guide/) in `keras`. Each layer is a function, which takes as input the previous layer.

In [ ]:
from keras.models import Model
from keras.layers import Input, Embedding
from keras.layers import Bidirectional, LSTM
from keras.layers import Dropout, Dense, Activation
import numpy as np

# Set a random seed for reproducibility
np.random.seed(42)

inputs = Input((MAX_LENGTH, ), 
               name='word_IDs')
embeddings = Embedding(input_dim=len(word2int), 
                       output_dim=128, 
                       mask_zero=True, 
                       name='embeddings')(inputs)
lstm = LSTM(units=256,
              return_sequences=True,
              name="LSTM")(embeddings)
dropout = Dropout(0.3, name='dropout')(lstm)
lstm_out = Dense(len(tag2int), name='output')(dropout)
output = Activation('softmax', name='softmax')(lstm_out)

model = Model(inputs=[inputs], outputs=[output])
model.summary()

In order to read the tags, the model needs 1-hot encoding, i.e., a vector with one dimension for each tag, all of them 0, except the one dimension corresponding to the tag we have. Lucikly, there exists a utility function for it.

In [ ]:
from keras.utils import to_categorical

train_tags_1hot = to_categorical(train_tags, len(tag2int))
test_tags_1hot = to_categorical(test_tags, len(tag2int))

# originally 50 tag IDs
print(train_tags[0])
# now 50 rows with 13 columns
print(train_tags_1hot[0].shape)
# the 1-hot encoding of tag ID 7
print(train_tags_1hot[0])

Now we can train the model

In [ ]:
batch_size = 16
epochs = 5

# compile the model we have defined above
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']
             )

# run training and capture ouput log
history = model.fit(train_sentences, train_tags_1hot,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.2)

We can plot the performance on training and dev set

In [ ]:
%matplotlib inline
import pandas as pd
import seaborn

df = pd.DataFrame(history.history)
df[['val_accuracy', 'accuracy']].plot.line();
df[['val_loss', 'loss']].plot.line();

Let's see how well we do on the test set

In [ ]:
loss, accuracy = model.evaluate(test_sentences, test_tags_1hot,
                       batch_size=batch_size, verbose=1)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Not too bad! Let's see how well it does with sentences that have ambiguous words:

In [ ]:
strange_sentences = ['Their management plan reforms worked'.split(),
                     "Attack was their best option".split()
                    ]
# convert to integers
strange_sentences_int = [[word2int.get(word, 1) for word in sentence] for sentence in strange_sentences]
# add padding
strange_sentences_int = pad_sequences(strange_sentences_int, maxlen=MAX_LENGTH, padding='post')

predictions = model.predict(strange_sentences_int)
print(predictions, predictions.shape)

Note some things:
1. The output for each word is a probability distribution over tags. Nice if we want it, but here, we are actually more interested in the one **best** tag for each token.
2. the tags are the integer IDs, so we need to translate them back
3. because we padded, the last item just gets repeated
    
We need to fix those before we have useful output:

In [ ]:
def inverse_transform(sentences, predictions):
    output = []
    for sentence, prediction in zip(sentences, predictions):
        # find the index of the highest-scoring tag and translate it back
        token_sequence = [int2tag[np.argmax(prediction[i])] for i in range(len(sentence))]
        output.append(token_sequence)
    return output

print(list(zip(strange_sentences, inverse_transform(strange_sentences, predictions))))

Looks ok! Let's see what we get if we process the sentence from both ends.

## Bi-LSTM

In [ ]:
from keras.layers import Bidirectional

# Set a random seed for reproducibility
np.random.seed(42)

inputs = Input((MAX_LENGTH, ), 
               name='word_IDs')
embeddings = Embedding(input_dim=len(word2int), 
                       output_dim=128, 
                       mask_zero=True, 
                       name='embeddings')(inputs)
#wrap the LSTM in a Bidirectional wrapper
bilstm = Bidirectional(LSTM(units=256, 
                            return_sequences=True), 
                       name="Bi-LSTM")(embeddings)
dropout = Dropout(0.3, name='dropout')(bilstm)
bilstm_out = Dense(len(tag2int), name='output')(dropout)
output = Activation('softmax', name='softmax')(bilstm_out)

model_bilstm = Model(inputs=[inputs], outputs=[output])
model_bilstm.summary()

We just added a casual 400k parameters to our model!

In [ ]:
batch_size = 16
epochs = 5

model_bilstm.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']
             )

history_bilstm = model_bilstm.fit(train_sentences, train_tags_1hot,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.2)

df = pd.DataFrame(history_bilstm.history)
df[['val_accuracy', 'accuracy']].plot.line();
df[['val_loss', 'loss']].plot.line();

In [ ]:
loss, accuracy = model_bilstm.evaluate(test_sentences, test_tags_1hot,
                       batch_size=batch_size, verbose=1)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

We are getting better on the test data, what about the confusing sentences?

In [ ]:
predictions_bilstm = model_bilstm.predict(strange_sentences_int)
print(list(zip(strange_sentences, inverse_transform(strange_sentences, predictions_bilstm))))

Not so much. We probably need a lot more data to prevent overfitting.

## Exercise

Get the large train, dev, and test data sets for CoNLL, apply the preprocessing steps and run the model on them. What performance do you get?

In [ ]:
# Your code here

## Sequential API Model

When you google for more details, you might come across the **Sequential API** (because we sequentially add layers, not because this is a sequence model). It does the same thing as the functional API, but in a different way. It is less flexible than the functional API we have been using, though.

In [ ]:
from keras.models import Sequential
from keras.layers import InputLayer
np.random.seed(42)

model_seq = Sequential()
model_seq.add(InputLayer(input_shape=(MAX_LENGTH, ), name="word_IDs"))
model_seq.add(Embedding(len(word2int), 128, mask_zero=True, name='embeddings'))
model_seq.add(Bidirectional(LSTM(256, return_sequences=True), name='bi-LSTM'))
model_seq.add(Dropout(0.3, name='dropout'))
model_seq.add(Dense(len(tag2int), name='output'))
model_seq.add(Activation('softmax', name='softmax'))
model_seq.summary()

In [ ]:
batch_size = 32
epochs = 5

model_seq.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model_seq.fit(train_sentences, train_tags_1hot,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.2)

In [ ]:
loss, accuracy = model_seq.evaluate(test_sentences, test_tags_1hot,
                       batch_size=batch_size, verbose=1)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

The sequential model does have a convenience function, though, `predict_classes()`, which we can use to get the tatg IDs. This changes our inverse transformation a bit, since we no longer need the argmax.

In [ ]:
predictions_seq = model_seq.predict_classes(strange_sentences_int)
for sentence, prediction in zip(strange_sentences, predictions_seq):
    pred_ = [int2tag[prediction[i]] for i in range(len(sentence))]
    print(sentence, pred_)
